# IMPORT LIBRARIES AND DATASET

In [ ]:
#@title Libraries
import tensorflow as tf
import tensorflow_hub as hub

import tensorflow_datasets as tfds

import time

from PIL import Image
import requests
from io import BytesIO

import matplotlib.pyplot as plt
import numpy as np

import os
import pathlib

In [ ]:
#@title Gradio
!pip install gradio --quiet
import gradio as gr

     |████████████████████████████████| 5.1 MB 5.2 MB/s 
     |████████████████████████████████| 2.0 MB 40.9 MB/s 
     |████████████████████████████████| 1.1 MB 55.5 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 140 kB 70.9 MB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 
     |████████████████████████████████| 256 kB 65.4 MB/s 
     |████████████████████████████████| 212 kB 77.8 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
     |████████████████████████████████| 144 kB 70.0 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
     |████████████████████████████████| 271 kB 67.0 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 11.1 MB 52.3 MB/s 
     |████████████████████████████████| 80 kB 10.0 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 4.0 MB 38.1 MB/s 
     |█████████████████████

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import image
import tensorflow as tf


In [ ]:
#@title Construct imagenet logit-to-class-name dictionary (imagenet_int_to_str)

!wget https://storage.googleapis.com/bit_models/ilsvrc2012_wordnet_lemmas.txt

imagenet_int_to_str = {}

with open('ilsvrc2012_wordnet_lemmas.txt', 'r') as f:
  for i in range(1000):
    row = f.readline()
    row = row.rstrip()
    imagenet_int_to_str.update({i: row})

--2022-06-17 07:09:38--  https://storage.googleapis.com/bit_models/ilsvrc2012_wordnet_lemmas.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.212.128, 173.194.214.128, 173.194.215.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21675 (21K) [text/plain]
Saving to: ‘ilsvrc2012_wordnet_lemmas.txt’

ilsvrc2012_wordnet_ 100%[===================>]  21.17K  --.-KB/s    in 0s      

2022-06-17 07:09:38 (95.5 MB/s) - ‘ilsvrc2012_wordnet_lemmas.txt’ saved [21675/21675]



In [ ]:
#@title Dùng TensorFlow Datasets để load bộ data oxford_flowers102
import tensorflow_datasets as tfds

(train, val, test), info = tfds.load('oxford_flowers102', 
                                      split=['train', 'validation', 'test'],
                                      shuffle_files=True,
                                      as_supervised=True, 
                                      with_info=True)
NUM_CLASSES = 102


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/oxford_flowers102/2.1.1.incompleteIDFBHF/oxford_flowers102-train.tfrecord


...

  0%|          | 0/6149 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/oxford_flowers102/2.1.1.incompleteIDFBHF/oxford_flowers102-validation.tfrecord


  0%|          | 0/1020 [00:00<?, ? examples/s]

Dataset oxford_flowers102 downloaded and prepared to /root/tensorflow_datasets/oxford_flowers102/2.1.1. Subsequent calls will reuse this data.


In [ ]:
#@title Lưu danh sách tên vào file txt
oxford_flowers_labels=[]
i=1
while i<102:
  oxford_flowers_labels.append(info.features['label'].names[i])
  i+=1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data_Preprocessing


In [ ]:
from tensorflow import cast, float32
from tensorflow.data.experimental import AUTOTUNE
from tensorflow import one_hot 
from tensorflow.image import resize

def preprocess_data(image, label):
  """
  Normalizes images: `uint8` -> `float32`.
  One hot encoding labels
  Resize to (224, 224)
  """


  return resize(cast(image, float32)/255. , [224, 224]), one_hot(label, 102)



In [ ]:
BATCH_SIZE = 102


In [ ]:
train_dataset = train.map(
    preprocess_data, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(info.splits['train'].num_examples)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(AUTOTUNE)

In [ ]:
val_dataset = val.map(
    preprocess_data, num_parallel_calls=AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.cache()
val_dataset = val_dataset.prefetch(AUTOTUNE)

In [ ]:
test_dataset = test.map(
    preprocess_data, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.cache()
test_dataset = test_dataset.prefetch(AUTOTUNE)

# Load the pre-trained BiT model

In [ ]:
# Load model into KerasLayer
model_url = "https://tfhub.dev/google/bit/m-r50x1/1"
module = hub.KerasLayer(model_url)

## Fine-tuning the BiT model
There are two steps:
1. Create a new model with a new final layer (which we call the ‘head’), and
2. Fine-tune this model using BiT-HyperRule, our hyperparameter heuristic.


In [ ]:
# Add new head to the BiT model

class MyBiTModel(tf.keras.Model):
  """BiT with a new head."""

  def __init__(self, num_classes, module):
    super().__init__()

    self.num_classes = num_classes
    self.head = tf.keras.layers.Dense(num_classes, kernel_initializer='zeros')
    self.bit_model = module
  
  def call(self, images):
    # No need to cut head off since we are using feature extractor model
    bit_embedding = self.bit_model(images)
    return self.head(bit_embedding)

model = MyBiTModel(num_classes=NUM_CLASSES, module=module)

#### Add new head to the BiT model

Since we want to use BiT on a new dataset (not the one it was trained on), we need to replace the final layer with one that has the correct number of output classes. This final layer is called the head.

Note that it is important to **initialise the new head to all zeros**.

### 1. Creating the new model



To create the new model, we:

1. Cut off the BiT model’s original head. This leaves us with the “pre-logits” output.
 - We do not have to do this if we use the ‘feature extractor’ models ((i.e. all those in subdirectories titled `feature_vectors`), since for those models the head has already been cut off.

2. Add a new head with the number of outputs equal to the number of classes of our new task. Note that it is important that we initialise the head to all zeroes.

### Fine-tuning loop

In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
#batch_size= 1020
# có 6149 images, 6149=11*53*13; 5549=13*43
# Create a callback that consists of earlystopping and checkpoint
ESCP_callback = [
    tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq= 'epoch'),
]


In [ ]:
"""
model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=['accuracy'])

# Fine-tune model
history = model.fit(
    train_dataset,
    batch_size=BATCH_SIZE,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks = [ESCP_callback],
    epochs= 8,  
    validation_data=test_dataset  
)
"""

Epoch 1/8
10/10 [==============================] - ETA: 0s - loss: 4.3435 - accuracy: 0.2627
Epoch 1: saving model to training_/cp-0001.ckpt
10/10 [==============================] - 84s 6s/step - loss: 4.3435 - accuracy: 0.2627 - val_loss: 3.5396 - val_accuracy: 0.8624
Epoch 2/8
10/10 [==============================] - ETA: 0s - loss: 2.6802 - accuracy: 0.9608
Epoch 2: saving model to training_/cp-0002.ckpt
10/10 [==============================] - 47s 5s/step - loss: 2.6802 - accuracy: 0.9608 - val_loss: 1.8499 - val_accuracy: 0.9615
Epoch 3/8
10/10 [==============================] - ETA: 0s - loss: 1.0991 - accuracy: 0.9912
Epoch 3: saving model to training_/cp-0003.ckpt
10/10 [==============================] - 47s 5s/step - loss: 1.0991 - accuracy: 0.9912 - val_loss: 0.8019 - val_accuracy: 0.9714
Epoch 4/8
10/10 [==============================] - ETA: 0s - loss: 0.4083 - accuracy: 0.9971
Epoch 4: saving model to training_/cp-0004.ckpt
10/10 [==============================] - 47s 5s/s

We see that our model attains over 98-99% training and validation accuracy.

# SAVE MODEL

In [ ]:
tf.keras.models.save_model(model, '/content/drive/MyDrive/Colab Notebooks/CAPSTONE PROJ/BiT fine_grained-17Jun')

# GRADIO

In [ ]:
# Load a previously trained model for gradio demonstration
model2=tf.keras.models.load_model('/content/drive/MyDrive/ColabNotebook/BiT fine_grained-17Jun-20220617T065450Z-001/BiT fine_grained-17Jun')

In [ ]:
model2.summary()

Model: "my_bi_t_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               multiple                  208998    
                                                                 
 keras_layer (KerasLayer)    multiple                  23500352  
                                                                 
Total params: 23,709,350
Trainable params: 208,998
Non-trainable params: 23,500,352
_________________________________________________________________


In [ ]:
with open('/content/flower_names.txt') as f:
    labels = f.readlines()

In [ ]:
from numpy import exp
def softmax(vector):
 e = exp(vector)
 return e / e.sum()

In [ ]:
def image_to_output (input_img):

 


  gr_img=[]
  gr_img.append(input_img)
  img2=resize(cast(gr_img, float32)/255. , [224, 224])

  #print(img2)

  x_test=np.asarray(img2)

  prediction = model2.predict(x_test,batch_size=1).flatten()
  prediction=softmax(prediction)

  confidences = {labels[i]: float(prediction[i]) for i in range(102)}
#  confidences = {labels[i]:float(top[i]) for i in range(num_predictions)}

  return confidences




In [ ]:
import gradio as gr

gr.Interface(fn=image_to_output, 
             inputs=gr.inputs.Image(shape=(224,224)),
             outputs=gr.outputs.Label(num_top_classes=5),
             interpretation="default"
             ).launch()

/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://59943.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f819ccb1a10>,
 'http://127.0.0.1:7861/',
 'https://59943.gradio.app')

## Acknowledgements

This colab is based on 

 BigTransfer (BiT): A step-by-step tutorial for state-of-the-art vision
By Jessica Yung and Joan Puigcerver